In [1]:
from scipy.fft import fft, ifft
from scipy import ndimage
import numpy as np
import cv2 as cv2
import matplotlib.pyplot as plt
import os

In [2]:
def GetImageROI(img, area = 0.1):    
    scale = np.sqrt(area)
    crop_h = int(img.shape[0] * scale)
    crop_w = int(img.shape[1] * scale)

    # Calculate start and end indices to crop the center
    start_row = (img.shape[0] - crop_h) // 2
    start_col = (img.shape[1] - crop_w) // 2
    end_row = start_row + crop_h
    end_col = start_col + crop_w

    # Crop the center 10% region
    center_crop = img[start_row:end_row, start_col:end_col]

    return center_crop.shape

In [3]:
def Calc_Image_Quality_Measure(img):
    img_copy  = img.copy()
    imgTff = fft(img_copy)
    imgShifted = np.fft.fftshift(imgTff)
    imgAbs = abs(imgShifted)
    imgFreq = np.fft.fftfreq(len(imgTff))
    mask = imgFreq >= 0
    positive_freqs = imgFreq[mask]
    positive_magnitudes = imgAbs[mask]

    maxfreq = positive_freqs[np.argmax(positive_magnitudes)]

    threshold = -maxfreq / 1000
    Th = np.sum(imgTff > threshold)
    num_rows = img_copy.shape[0]
    num_col = img_copy.shape[1]

    Iqm = Th / (num_rows*num_col)
    return Iqm


In [4]:
def AnalizeImage(img, ROIarea = 0.1):
    image_IQM = Calc_Image_Quality_Measure(img)
    imageROI_IQM_05 = Calc_Image_Quality_Measure(GetImageROI(img, 0.05))
    imageROI_IQM_1 = Calc_Image_Quality_Measure(GetImageROI(img))

    print("IQM original:", image_IQM, "ROI 0.05 IQM:", imageROI_IQM_05, "ROI 0.1 IQM:", imageROI_IQM_1)
    return [image_IQM, imageROI_IQM_05, imageROI_IQM_1]

In [ ]:
video_path = 'TP2\focus_video.mov'
output_folder = 'TP2\Images'

os.makedirs(output_folder, exist_ok=True)

# Open video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")

frame_count = 0
success = True

while success:
    success, frame = cap.read()
    if success:
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames to '{output_folder}'")

Error: Could not open video.
Extracted 0 frames to 'TP2\Images'


In [6]:
print("Video path:", os.path.abspath(video_path))

if not os.path.exists(video_path):
    print("❌ File not found at path:", video_path)
else:
    print("✅ File found.")

Video path: e:\importante\Master\MLOps\TP\TP2\Computer-Vision-1\TP2ocus_video.mov
❌ File not found at path: TP2ocus_video.mov
